# 4. Neural Networks Learning

- 작성자: [JWarmenhoven's Exercise 4](https://nbviewer.jupyter.org/github/JWarmenhoven/Coursera-Machine-Learning/blob/master/notebooks/Programming%20Exercise%204%20-%20Neural%20Networks%20Learning.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# load MATLAB files
from scipy.io import loadmat

# style setting
%matplotlib inline
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

#### load MATRAB data

In [3]:
data = loadmat("/content/drive/My Drive/study/ML(Andrew Ng)/data/ex4data1.mat")
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [4]:
weights = loadmat("/content/drive/My Drive/study/ML(Andrew Ng)/data/ex3weights.mat")
weights.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [5]:
y = data["y"]
X = np.c_[np.ones((data["X"].shape[0], 1)), data["X"]]

In [6]:
print("X:", X.shape, "(with intercept)")
print("y:", y.shape)

X: (5000, 401) (with intercept)
y: (5000, 1)


In [7]:
theta1 = weights["Theta1"];  theta2 = weights["Theta2"]
params = np.r_[theta1.ravel(), theta2.ravel()]

print("theta1:", theta1.shape)
print("theta2:", theta2.shape)
print("params:", params.shape)

theta1: (25, 401)
theta2: (10, 26)
params: (10285,)


### Neural Network

- input layer: 20 X 20
- Hidden layer: 25
- number of labels: 10

sigmoid

In [8]:
def sigmoid(z):
  return (1 / (1 + np.exp(-z)))

In [9]:
def sigmoidGradient(z):
  return (sigmoid(z) * (1 - sigmoid(z)))

**logistic hypothesis + regularization(not lasso)**

In [10]:
def nnCostFunction(params, input_size, hidden_size, num_labels, features, classes, reg):
  theta1 = params[0:(hidden_size * (input_size + 1))].reshape(hidden_size, (input_size + 1))
  theta2 = params[(hidden_size * (input_size + 1)):].reshape(num_labels, (hidden_size + 1))

  m = features.shape[0]
  y_matrix = pd.get_dummies(classes.ravel())

  a1 = features         # 5000 X 401
  z2 = theta1.dot(a1.T) # (25 X 401) * (401 * 5000) = 25 X 5000
  a2 = np.c_[np.ones((features.shape[0], 1)), sigmoid(z2.T)]  # 5000 X 26, included bias unit
  z3 = theta2.dot(a2.T) # (10 X 26) * (26 X 5000) = 10 X 5000
  a3 = sigmoid(z3)      # 10 X 5000

  J = -(1/m) * np.sum(np.sum(np.log(a3.T)*(y_matrix) + np.log(1 - a3).T*(1 - y_matrix))) + \
      (reg / (2*m)) * (np.sum(np.square(theta1[:, 1:])) + np.sum(np.square(theta2[:, 1:])))
  # sum((5000 X 10) * (y_matrix, 5000 X 10) + (5000 X 10) * (5000 X 10))

  # Gradients
  d3 = a3.T - y_matrix  # 5000 X 10
  d2 = theta2[:, 1:].T.dot(d3.T) * sigmoidGradient(z2)
  # ((25 X 10) * (10 X 5000)) * (25 * 5000) = (25 * 5000)
  # .* 같은 건가? matrix적으론 안 되는 건데?

  delta1 = d2.dot(a1)   # (25 X 5000) * (5000 X 401) = 25 X 401
  delta2 = d3.T.dot(a2) # (10 X 5000) * (5000 X 26) = 10 X 26

  theta1_ = np.c_[np.ones((theta1.shape[0], 1)), theta1[:, 1:]]
  theta2_ = np.c_[np.ones((theta2.shape[0], 1)), theta2[:, 1:]]
  theta1_grad = (delta1 + (theta1_ * reg))/m
  theta2_grad = (delta2 + (theta2_ * reg))/m

  return (J, theta1_grad, theta2_grad)

In [11]:
nnCostFunction(params, 400, 25, 10, X, y, 0)[0]

0.28762916516131876

In [12]:
nnCostFunction(params, 400, 25, 10, X, y, 1)[0]

0.3837698590909235

In [13]:
[sigmoidGradient(z) for z in [-1, -0.5, 0, 0.5, 1]]

[0.19661193324148185,
 0.2350037122015945,
 0.25,
 0.2350037122015945,
 0.19661193324148185]